In [29]:
from pathlib import Path
import re
import math
from collections import Counter
import json

In [37]:
cdir = Path('.')
wasmdir = cdir / 'wasms'
profdir = cdir / 'profiles'

#def parse_obj(fname):
#    with open(fname) as f:
#        content = f.read()
#    p = re.findall(r'Custom:\n - name: "name"\n(.*)Custom:\n', content, flags=re.DOTALL)[0]
#    num_imports = int(re.findall(r'Import\[(\d+)\]:', content)[0])
#    matches = re.findall(r'\s*-\s*func\[(\d+)\]\s*<(\S+)>', p, flags=re.DOTALL)
#    return {int(k): v for k, v in dict(matches).items() if int(k) >= num_imports}
    

agg_metric = Counter({})
agg_cc = {}
share_ftr = {}

num_files = 0
for wasm in wasmdir.glob('*.wasm'):
    print(wasm)
    with open(profdir / f"{wasm.stem}.profile") as f:
        # Get profile counts
        count_vec = [int(x.split(':')[1]) for x in f.read().split(',')]
        with open(profdir / f"{wasm.stem}.func_idx") as g:
            fn_dict = json.load(g)['defs']
        # Log normalize the counts
        app_profile = {k: (math.log1p(count_vec[v])) for k, v in fn_dict.items()}
        app_profile = {k: v for k, v in sorted(app_profile.items(), key=lambda item: item[1], reverse=True)}
        norm = sum(app_profile.values())
        app_profile = {k: round(v/norm, 4) for k, v in app_profile.items() }
        with open(cdir / 'parsed' / f"{wasm.stem}.json", 'w') as fp:
            json.dump(app_profile, fp, indent=4)
        # Compute share factor 
        for k in app_profile.keys():
            if k not in share_ftr:
                share_ftr[k] = -1
            share_ftr[k] += 1
        # Aggregate stats across all files
        agg_metric.update(Counter(app_profile))
        num_files += 1
        # Call counts
        with open(profdir / f"{wasm.stem}.call_count") as g:
            agg_cc = { **agg_cc, **json.load(g) }
    
# Apply aggregation formula
constant = 1500
agg_metric = {k: constant * (v/num_files)*(share_ftr[k]**3) for k, v in agg_metric.items() if k in agg_cc}
outd = {k: {"metric": round(v, 4), \
            "share": share_ftr[k], \
            # -1 for calls since we include the call to instrument
            "calls": agg_cc[k]["calls"] - 1, \
            "indirects": agg_cc[k]["indirects"] \
           } for k, v in sorted(agg_metric.items(), key=lambda item: item[1], reverse=True)}

with open(cdir / 'aggregate.json', 'w') as f:
    json.dump(outd, f, indent=4)


wasms/paho-mqtt-app.wasm
wasms/lua.wasm
wasms/bash.wasm
wasms/vim.wasm


In [58]:
with open(cdir / 'aggregate.json', 'r') as f:
    dt = json.load(f)
dt = {k: v for k, v in dt.items() if v['metric'] != 0}  

In [63]:
# Total functions
p = dt
print(len(p))

251


In [64]:
# No call functions
p = {k: v['metric'] for k, v in dt.items() if v['calls'] == 0 and v['indirects'] == 0}
print(len(p))
print(p)

42
{'a_ctz_32': 200.475, 'memcpy': 174.15, 'strlen': 159.975, 'a_or': 156.9375, 'strncmp': 145.8, '__stpcpy': 143.775, '__towrite': 118.4625, 'strcmp': 118.4625, 'memset': 111.375, 'strspn': 93.15, 'memchr': 86.0625, 'memcmp': 83.025, 'a_swap': 79.9875, '__pthread_setcancelstate': 73.9125, '__errno_location': 71.8875, 'isalnum': 64.8, 'tolower': 63.7875, 'a_or_64': 48.6, 'a_clz_32': 43.5375, '__shlim': 40.5, 'a_store': 36.45, '__wasm_init_memory': 24.3, 'a_ctz_64': 18.3, 'a_fetch_add': 13.1625, 'a_barrier': 12.9, 'sigemptyset': 12.9, 'dummy': 10.125, '__testcancel': 7.5, '__memrchr': 7.2, 'a_cas': 6.075, 'dummy_0': 5.7, 'toupper': 4.8, 'a_inc': 3.9, 'sigismember': 3.6, 'htons': 3.0, 'atoi': 3.0, 'setvbuf': 2.1, 'a_dec': 1.8, 'rand': 1.425, 'abs': 1.0875, 'sem_destroy': 0.7125, 'srand': 0.225}


In [65]:
# Only direct calls
p = {k: v['metric'] for k, v in dt.items() if v['calls'] != 0 and v['indirects'] == 0}
print(len(p))
print(p)

193
{'__unlock': 205.5375, 'enframe': 202.5, '__libc_malloc_impl': 200.475, 'get_meta': 192.375, '__libc_free': 192.375, 'free': 191.3625, 'alloc_slot': 166.05, 'nontrivial_free': 163.0125, 'default_malloc': 152.8875, '__syscall_ret': 149.85, '__malloc_alloc_meta': 144.7875, 'strcpy': 143.775, '__libc_realloc': 139.725, 'realloc': 139.725, 'printf_core': 137.7, 'pop_arg': 131.625, '__strchrnul': 125.55, 'sn_write': 118.4625, 'vsnprintf': 116.4375, '__clock_gettime': 108.3375, '__lock': 100.2375, 'strchr': 96.1875, '__mmap': 94.1625, 'strdup': 92.1375, 'strtok': 91.125, 'strcspn': 91.125, 'putenv': 90.1125, '__stdio_write': 84.0375, '__munmap': 83.025, 'sccp': 70.875, '__syscall_cp': 70.875, 'strnlen': 67.8375, 'snprintf': 67.8375, 'getenv': 60.75, 'time': 59.7375, '__libc_sigaction': 51.6375, '__sigaction': 51.6375, 'sigaddset': 47.5875, '__fstatat': 46.575, 'read': 41.5125, 'printf': 40.5, '__lockfile': 37.4625, '__unlockfile': 37.4625, '__intscan': 37.4625, '__stdio_read': 37.4625, '

In [66]:
# Only indirect calls
p = {k: v['metric'] for k, v in dt.items() if v['calls'] == v['indirects'] and v['calls'] != 0}
print(len(p))
print(p)

3
{'__toread': 38.475, 'wrapper_cmp': 25.5, '__fclose_ca': 1.8}


In [67]:
# Both direct and indirect calls
p = {k: v['metric'] for k, v in dt.items() if v['indirects'] != 0 and v['indirects'] != v['calls']}
print(len(p))
print(p)

13
{'__fwritex': 160.9875, 'vfprintf': 122.5125, 'fflush': 39.4875, '__uflow': 34.425, 'fwrite': 22.275, 'sift': 19.2, 'fclose': 18.225, 'trinkle': 17.1, '__overflow': 10.2, 'fread': 7.0875, '_pthread_cleanup_pop': 1.8, 'libc_exit_fini': 1.0125, '__stdio_exit': 1.0125}
